# Neural network training on csv data
After running the first cell, you will have to sign in with google and copy a code

In [1]:
!pip install -Uqq fastbook
import fastbook
from fastbook import *
fastbook.setup_book()
from fastai.tabular.all import *

4/1AY0e-g78i5JhWMyhJKzEuQj27ZNVB775VBwK0gmdSMjFz0cqkgK96LpvtIoMounted at /content/gdrive


Replace the `path` variable with a relative path, or one relative to `gdrive` to access data from Google Drive

In [4]:
# following tutorial from: https://docs.fast.ai/tutorial.tabular.html
path = Path("gdrive/MyDrive/ParkinsonsData")
path.ls()

(#1) [Path('gdrive/MyDrive/ParkinsonsData/DadDoubleSupport - AiDadMom-2.csv')]

In [5]:
# change the name of the csv inside of the path
df = pd.read_csv(path/'DadDoubleSupport - AiDadMom-2.csv')
df.head()

,sourceName,double,speed,length,Asym
0,1,0.305,2.550107,25.590551,0.0
1,1,0.294,2.930387,28.346457,0.0
2,1,0.293,3.042233,28.740157,0.0
3,1,0.292,2.997495,28.740157,0.0
4,1,0.291,2.997495,29.133858,0.0


In [22]:
# update categorical names (one or the other), and change y_names to be the column you want to predict
# also change path and csv name
dls = TabularDataLoaders.from_csv(path/'DadDoubleSupport - AiDadMom-2.csv', path=path, y_names="sourceName",
    cont_names = ['double', 'speed', 'length', 'Asym'],
    procs = [FillMissing, Normalize])

In [23]:
# visualize data before training
dls.show_batch()

,double,speed,length,Asym,sourceName
0,0.298,3.758053,31.889763,5.225935e-10,1.0
1,0.281,3.422513,38.582676,5.225935e-10,1.0
2,0.328,2.639585,25.590551,2.000000e-02,2.0
3,0.279,3.377774,30.314960,5.225935e-10,1.0
4,0.284,3.310666,31.889763,5.225935e-10,1.0
5,0.297,2.013243,24.409449,5.225935e-10,2.0
6,0.298,2.080351,22.047245,5.225935e-10,1.0
7,0.322,1.632964,18.110237,5.225935e-10,1.0
8,0.244,1.789549,27.165354,5.225935e-10,1.0
9,0.325,2.661954,24.409449,3.000000e-02,1.0


In [24]:
# create a model
learn = tabular_learner(dls, metrics=mse)

In [25]:
# fit for one "epoch" or entire loop of all of the data (change the number and see what happens!)
learn.fit_one_cycle(30)

epoch,train_loss,valid_loss,mse,time
0,1.526082,1.159315,1.159315,00:00
1,0.989534,0.412356,0.412356,00:00
2,0.526668,0.105192,0.105192,00:00
3,0.285440,0.082447,0.082447,00:00
4,0.173317,0.063393,0.063393,00:00
5,0.118458,0.067020,0.067020,00:00
6,0.092204,0.054830,0.054830,00:00
7,0.080560,0.056012,0.056012,00:00
8,0.072744,0.056965,0.056965,00:00
9,0.066846,0.054213,0.054213,00:00


In [27]:
# see predicted vs real results in new column
learn.show_results()

,double,speed,length,Asym,sourceName,sourceName_pred
0,0.613338,-0.469268,-0.477593,0.455094,1.0,1.310353
1,-2.217011,-0.281288,-0.834445,-0.219987,1.0,1.047540
2,-0.630301,1.034565,0.949812,-0.219987,1.0,0.992753
3,-0.801836,1.899269,1.841941,-0.219987,1.0,0.991121
4,-0.544533,-0.657247,-0.477593,-0.219987,1.0,0.988127
5,-0.458764,0.696202,0.682174,-0.219987,1.0,0.976073
6,-0.758953,0.996969,0.949812,-0.219987,1.0,0.987058
7,0.913526,0.921777,1.395877,-0.219987,1.0,0.967394
8,-0.973373,0.207457,0.057684,0.590111,1.0,1.039067


In [28]:
# predict on a row in df (in this case, row 0)
row, clas, probs = learn.predict(df.iloc[0])
row.show()

,double,speed,length,Asym,sourceName
0,0.305,2.550107,25.590551,5.225935e-10,1.108266


In [34]:
# get predictions on a new_dataset using the trained model
# new_dataset should have same column names as the one you trained the model on
new_dataset = df.copy()
new_dataset.drop(['sourceName'], axis=1, inplace=True)
new_dataset.head()

,double,speed,length,Asym
0,0.305,2.550107,25.590551,0.0
1,0.294,2.930387,28.346457,0.0
2,0.293,3.042233,28.740157,0.0
3,0.292,2.997495,28.740157,0.0
4,0.291,2.997495,29.133858,0.0


In [65]:
# get predictions tensor
dl = learn.dls.test_dl(new_dataset)
pred_tensor = learn.get_preds(dl=dl)
pred_tensor

(tensor([[1.1083],
         [1.0224],
         [0.9858],
         ...,
         [0.9577],
         [1.0078],
         [1.1294]]), None)

In [72]:
b = [a.numpy()[0] for a in list(pred_tensor[0])]
predictions = [1 if a < 1.5 else 2 for a in b]
predictions[:10]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [76]:
new_dataset['pred_sourceName'] = pd.Series(predictions)

In [77]:
# original, for comparison
df.head()

,sourceName,double,speed,length,Asym
0,1,0.305,2.550107,25.590551,0.0
1,1,0.294,2.930387,28.346457,0.0
2,1,0.293,3.042233,28.740157,0.0
3,1,0.292,2.997495,28.740157,0.0
4,1,0.291,2.997495,29.133858,0.0


In [78]:
# new predicted
new_dataset.head()

,double,speed,length,Asym,a,predicted,pred_sourceName
0,0.305,2.550107,25.590551,0.0,"(tensor(1.1083),)",1,1
1,0.294,2.930387,28.346457,0.0,"(tensor(1.0224),)",1,1
2,0.293,3.042233,28.740157,0.0,"(tensor(0.9858),)",1,1
3,0.292,2.997495,28.740157,0.0,"(tensor(1.0046),)",1,1
4,0.291,2.997495,29.133858,0.0,"(tensor(0.9973),)",1,1
